<a href="https://colab.research.google.com/github/Isakimie/CREWAI_SistemaMultiAgentes_de_viagens/blob/main/Guia_Isa_CREWAI_SistemaMultiAgentes_de_viagens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**29/11/25 São Paulo
WoMakersCode e FIAP - Criando agentes AI**

In [ ]:
!pip install --quiet crewai

In [ ]:
from crewai import Agent, Task, Crew
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [ ]:
from crewai import LLM

gemini_llm = LLM(
    model="gemini/gemini-2.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=0.0
)

In [ ]:
guia_turistico = Agent(
    role="Guia Turistico",
    goal="Entregar um roteiro de viagem para {destino} com o melhor custo benefício dado tempo {tempo} de férias",
    backstory="""
    Você é um guia turístico nativo com mais de 10 anos de experiência recebendo turista de todo o mundo, foca em viagem baixo custo,
    indica os lugares mais famosos para turistas, mas também indica locais para ir e comer como um nativo.
    """,
    llm=gemini_llm
)

In [ ]:
escrever_roteiro = Task(
    agent=guia_turistico,
    description="""
    Escreva um roteiro de viagem para {destino}.
    Considere as melhores opções de restaurante considerando o melhor custo benefício.
    Inclua também opções de experiências nativas do local {destino}.
    E encaixe tudo no tempo disponível que é {tempo}.
    """,
    expected_output="Lista de restaurantes e experiências com horários definidos para cada um."
)

In [ ]:

crew = Crew(
    agents=[guia_turistico],
    tasks=[escrever_roteiro],
    verbose=True
)

In [ ]:

payload = {
    "destino": "São Paulo",
    "tempo": "4 dias"
}

roteiro_pronto = crew.kickoff(inputs=payload)

In [ ]:
!pip install --quiet crewai[tools]

In [ ]:
from crewai_tools import SerperDevTool
import os

search_tool = SerperDevTool()
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [ ]:
guia_turistico_mcqueen = Agent(
    role="Guia Turistico",
    goal="Entregar um roteiro de viagem para {destino} com o melhor custo benefício dado tempo {tempo} de férias",
    backstory="""
    Você é um guia turístico nativo com mais de 10 anos de experiência recebendo turista de todo o mundo, foca em viagem baixo custo,
    indica os lugares mais famosos para turistas, mas também indica locais para ir e comer como um nativo.
    """,
    llm=gemini_llm,
    tools=[search_tool]
)

In [ ]:
guia_gastronomico_mcqueen = Agent(
    role="Guia Gastronomico",
    goal="""
    Entregar um roteiro de restaurantes em {destino} com as melhores indicações e lugares atualizados.
    Pesquise na internet restaurantes com melhor custo benefício em {destino}
    dado tempo {tempo} de férias.
    """,
    backstory="Você é um guia gastronomico com 10 anos de experiência especialista em comida regional.",
    llm=gemini_llm,
    tools=[search_tool]
)

In [ ]:
agenciador_mcqueen = Agent(
    role="Agenciador de Viagens",
    goal="""
    Juntar os resultados do Guia Gastronomico e do Guia Turistico, para entregar um roteiro de viagens com o melhor custo benefício.
    Não invente nada, não acrescente informações novas.
    """,
    backstory="Você é um agenciador de viagens com 10 anos de experiência especialista em viagens solo.",
    llm=gemini_llm,
    tools=[search_tool]
)

In [ ]:
from crewai import Task
escrever_roteiro_mcqueen = Task(
    agent=guia_turistico_mcqueen,
    description="""
    Escreva um roteiro de viagem para {destino}.
    Considere as melhores opções de airbnb, hostel e hotel e pesquise na internet o melhor custo benefício.
    Pesquisa também opções de experiências nativas do local {destino}.
    E encaixe tudo no tempo disponível que é {tempo}.
    """,
    expected_output="Roteiro de viagem com praias e experiências com horários definidos para cada um.",
    tools=[search_tool]
)

In [ ]:
escrever_roteiro_gastronomico_mcqueen = Task(
    agent=guia_gastronomico_mcqueen,
    description="""
    Escreva um roteiro gastronomico para {destino}.
    Considere as melhores opções de restaurante pesquise na internet o melhor custo benefício.
    Considere apenas locais com reviews acima de 3,5 estrelas.
    """,
    expected_output="Lista de restaurantes com bom custo benefício e com comida regional.",
    tools=[search_tool]
)

In [ ]:
escrever_roteiro_final = Task(
    agent=agenciador_mcqueen,
    description="""
    Junte e estruture o resultado dos agentes Guia Turistico e Guia gastronomico para entregar um roteiro de viagem direto e prático.
    Encaixe os horarios, e se for preciso, reorganize os horarios para entregar uma experiencia completa e satisfatória em {destino}.
    """,
    expected_output="Roteiro completo para {destino} com horários e locais direto e prático.",
)

In [ ]:
crew_mcqueen = Crew(
    agents=[guia_turistico_mcqueen, guia_gastronomico_mcqueen, agenciador_mcqueen],
    tasks=[escrever_roteiro_mcqueen, escrever_roteiro_gastronomico_mcqueen, escrever_roteiro_final],
    verbose=True
)

In [ ]:
payload_mcqueen = {
    "destino": "São Paulo",
    "tempo": "04 dias"
}

roteiro_mcqueen_pronto = crew_mcqueen.kickoff(inputs=payload_mcqueen)